In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1604158/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1604158/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_10_5,0.490758,-0.033678,-0.769389,-7.920593,0.486431,2.072204,1.304770,0.688930,1.559172e+14,0.697446,-0.051339,0.708430,67.441319,139.196099,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_11_1,0.581876,-0.056853,-0.815627,0.705872,0.399395,2.118663,1.338867,0.606871,1.564466e+14,0.631977,0.136776,0.641930,67.835610,139.590390,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_11_6,0.601130,-0.071960,-0.872564,0.679286,0.381003,2.148948,1.380853,0.661727,1.616777e+14,0.617254,0.176527,0.626975,67.929898,139.684678,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_10_2,0.497682,-0.076930,-0.988384,-6.336039,0.479817,2.158910,1.466260,0.566556,1.615345e+14,0.692688,-0.037043,0.703597,67.468701,139.223480,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_25_7,0.658581,-0.092700,-1.634871,0.596888,0.326125,2.190523,1.942988,0.712830,1.368425e+14,0.571074,0.295135,0.580068,68.240947,139.995727,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,model_3_21_2,0.725321,-23.064307,-57.128863,0.663642,0.262375,48.241466,42.864977,0.235763,1.006954e+14,0.512226,-0.171965,0.520293,100.675958,207.220934,"Hidden Size=[12], regularizer=0.02, learning_r..."
220,model_3_21_0,0.725984,-24.250822,-58.877007,0.699370,0.261742,50.620061,44.154081,0.210720,1.033248e+14,0.511607,-0.169134,0.519664,100.680796,207.225772,"Hidden Size=[12], regularizer=0.02, learning_r..."
221,model_2_47_8,0.588989,-100.345930,-155.828063,0.537091,0.392600,203.167137,115.647047,0.757317,1.848682e+14,0.626578,0.151461,0.636446,67.869927,139.624707,"Hidden Size=[8], regularizer=0.02, learning_ra..."
222,model_2_47_5,0.528413,-103.016446,-20.439887,0.548570,0.450462,208.520693,15.810051,0.738538,1.657984e+14,0.671165,0.026402,0.681735,67.594961,139.349741,"Hidden Size=[8], regularizer=0.02, learning_ra..."
